In [1]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx
!pip install dbfread


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 47.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# We will work on New Hampshire
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).
The NJ data is formatted the same way, just replace 'nh' with 'nj'

### This is the current assignment of precinct to congressional districts (2 of them for NH)

In [ ]:
nh_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nh_current_assignment

FileNotFoundError: [Errno 2] No such file or directory: 'Map_Data/precinct-assignments-congress-nh.csv'

### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [ ]:
nh_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nh.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nh_precinct_data = nh_precinct_data[keepcolumns]
nh_precinct_data

### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [ ]:
shpfile = 'Map_Data/nh_vtd_2020_bound/nh_vtd_2020_bound.shp'
dbffile = 'Map_Data/nh_vtd_2020_bound/nh_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nh_vtd_2020_bound/nh_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nh_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nh_precinct_boundaries[rec[3]]=geom

In [ ]:
print(shxfile)

### Example: Town of Barrington precinct

In [ ]:
#See for instance the precinct boundaries for the town of Barrington precinct
print(Polygon(shape(nh_precinct_boundaries['33003SAND01'])))

In [ ]:
# We can see the coordinates of the bouding box of the district
print(Polygon(shape(nh_precinct_boundaries['33017BARR01'])).bounds)
# Or we can calculate the geographical centroid of the district
print(Polygon(shape(nh_precinct_boundaries['33017BARR01'])).centroid)

In [ ]:
# We can also get the data from that district from our data table
nh_precinct_data[nh_precinct_data['GEOID20']=='33017BARR01']

In [ ]:
# And the current assignment of the district
nh_current_assignment[nh_current_assignment['GEOID20']=='33017BARR01']

# How to get a district by name 

In [ ]:

#The name information is in the dbf file. You need the dbfread library to convert it to a dataframe
from dbfread import DBF

dbf = DBF(dbffile)
nh_names_df = pd.DataFrame(iter(DBF(dbffile)))
nh_names_df

In [ ]:
nh_names_df[nh_names_df['NAME20'].str.contains('WILTON')]

# Some possible redistricting strategies

### 1. Geographical

Let's start by creating simple geopgraphical maps, splitting the district in half North/South, or East/West.
New Hampshire's bounding box is (-72.557247,42.69699,-70.610621,45.305476) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state though the middle: everything west of longitude -71.583934 is in District 1, everything east is in District 2. We will use the precinct centroids to assign them.
Import the Map to DRA to look at it.

In [ ]:
nh_longitude_assignment = nh_current_assignment.copy()
nh_longitude_assignment['District'] = 0
for index, row in nh_longitude_assignment.iterrows():
    try:
        if shape(nh_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
            centroid = Polygon(shape(nh_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nh_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nh_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nh_precinct_boundaries[row['GEOID20']]).type)
            pass
        if centroid.x <= -71.583934:
            nh_longitude_assignment.iloc[index,nh_longitude_assignment.columns.get_loc('District')] = 1
        else:
            nh_longitude_assignment.iloc[index,nh_longitude_assignment.columns.get_loc('District')] = 2
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nh_longitude_assignment.to_csv('Recitation maps/nh_centerlongitude_map.csv',index=False)

Let's now split the state though the middle horizontally: everything north of latitude 44.001233 is in District 1, everything south is in District 2. We will use the precinct centroids to assign them. 
Import the Map to DRA to look at it.

In [ ]:
nh_latitude_assignment = nh_current_assignment.copy()
nh_latitude_assignment['District'] = 0
for index, row in nh_latitude_assignment.iterrows():
    try:
        if shape(nh_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
            centroid = Polygon(shape(nh_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nh_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nh_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nh_precinct_boundaries[row['GEOID20']]).type)
            pass
        if centroid.y <= 44.001233:
            nh_latitude_assignment.iloc[index,nh_latitude_assignment.columns.get_loc('District')] = 1
        else:
            nh_latitude_assignment.iloc[index,nh_latitude_assignment.columns.get_loc('District')] = 2
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nh_latitude_assignment.to_csv('Recitation maps/nh_centerlatitude_map.csv',index=False)

### Look at the statistics for the maps
Are they good maps?
The population are completely different. We can redraw the maps using different split longitude (resp. latitude) values. You can code a binary search that stops when the two populations are within 5% for example.

(There are four district for which the shapely data is not included, unfortunately this is from the census data so we will have to fix it manually later. Two of these have no population so are easy to ignore and just assign to the neighboring district. Two:  33013CONC05 and 33017SOME03 are populated district. You can hardcode their assignment for now.)

In [ ]:
# To get the population of a District:
D1_list = nh_latitude_assignment[nh_latitude_assignment['District']==1]['GEOID20']
D2_list = nh_latitude_assignment[nh_latitude_assignment['District']==2]['GEOID20']
popD1= nh_precinct_data[nh_precinct_data['GEOID20'].isin(D1_list)]['Total_2020_Total'].sum()
popD2= nh_precinct_data[nh_precinct_data['GEOID20'].isin(D2_list)]['Total_2020_Total'].sum()
print('D1 pop = '+str(popD1)+', D2 pop = '+str(popD2))

Let's write a function that gives us the population of the districts (returns a dictionary)

In [ ]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

Let's see the split at Latitude = 43

In [ ]:
nh_latitude43_assignment = nh_current_assignment.copy()
nh_latitude43_assignment['District'] = 0
for index, row in nh_latitude43_assignment.iterrows():
    try:
        if shape(nh_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
            centroid = Polygon(shape(nh_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nh_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nh_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nh_precinct_boundaries[row['GEOID20']]).type)
            pass
        if centroid.y <= 43:
            nh_latitude43_assignment.iloc[index,nh_latitude43_assignment.columns.get_loc('District')] = 1
        else:
            nh_latitude43_assignment.iloc[index,nh_latitude_assignment.columns.get_loc('District')] = 2
    except KeyError: 
        pass
nh_latitude43_assignment.to_csv('Recitation maps/nh_latitude43_map.csv',index=False)
getDistrictPopulations(nh_latitude43_assignment,nh_precinct_data,2)

Import the map to DRA. 

### 2. Graph-based

We can see the map as a graph, each precinct is a vertex, with edges drawn between neighboring precincts. We will use the NetworkX graph library. This will allow us to check for connectivity (contiguity) of the districts.

First, we create a district graph. For this, we use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nh.csv

In [ ]:
nh_contiguity = pd.read_csv('Contiguity_nh.csv', header=None)
nh_contiguity.columns = ['Precinct','Neighbors']
admin_water_precincts =['33017SOME03','33015ZZZZZZ']
#Two precinct are administrative, have no population, and while they should be included in the assignment, they cannot be used for contiguity

# Let's graph the district from the nh_latitude43_assignment map
district1_graph = nx.Graph() #creates an empty undirected graph
district1_nodes = nh_latitude43_assignment[nh_latitude43_assignment['District']==1]['GEOID20']
district1_graph.add_nodes_from(district1_nodes)
for id in district1_nodes:
    neighbors = ast.literal_eval(nh_contiguity[nh_contiguity['Precinct']==id]['Neighbors'].values.tolist()[0])
    # needed to convert string to list because the csv encodes the list as a string
    for neighbor in neighbors:
        district1_graph.add_edge(id,neighbor)

print(nx.is_connected(district1_graph))


Now let's add a disconnected precinct to the district

In [ ]:
district1_graph.add_node("33007JEFF01")
neighbors = nh_contiguity[nh_contiguity['Precinct']=="33007JEFF01"]['Neighbors'].values.tolist()[0].replace('[','').replace(']','').replace(' ','').split(',')
for neighbor in neighbors:
        district1_graph.add_edge(id,neighbor)
print(nx.is_connected(district1_graph))


So we can use the nx.is_connected function to check for contiguity. Here is a general function.

In [ ]:
def isDistrictContiguous(district_num, assignment, contiguity_list, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    return nx.is_connected(district_graph)

In [ ]:
isDistrictContiguous(2,nh_latitude43_assignment,nh_contiguity,admin_water_precincts)

#### Flood Fill Algorithm
Here we will use a simple flood fill algorithm which will assign half of the precinct to D2 based on a BFS traversal.
It does not consider population, but does check that both districts stay contiguous


In [ ]:
nh_floodfill_assignment = nh_current_assignment.copy()
nh_floodfill_assignment['District'] = 2 # Let's assign everything to D2, and build D1 via flood-fill.

d1_size = 0

next_precincts = ["33005HINS01"] #this is the district on the bottom left of the map. You can start anywhere else.
visited = []

while d1_size<163: #(there are 326 precincts total. We want to assign half to D1)
    nh_floodfill_assignment.loc[nh_floodfill_assignment['GEOID20']==next_precincts[0],'District']=1
    neighbors = ast.literal_eval(nh_contiguity[nh_contiguity['Precinct']==next_precincts[0]]['Neighbors'].values.tolist()[0])
    for neighbor in neighbors:
        #print(nh_floodfill_assignment[nh_floodfill_assignment['GEOID20']==neighbor]['District'].values[0])
        #if nh_floodfill_assignment[nh_floodfill_assignment['GEOID20']==neighbor]['District'].values[0]!=1: #only add precinct we haven't visited
        if neighbor not in visited and neighbor not in next_precincts : #only add if not visited and not already in list
            next_precincts.append(neighbor)
    d1_size += 1
    visited.append(next_precincts[0])
    next_precincts.pop(0)
    
    

print(isDistrictContiguous(1,nh_floodfill_assignment,nh_contiguity))
print(isDistrictContiguous(2,nh_floodfill_assignment,nh_contiguity))
# Note: in some cases flood fill breaks contiguity, it may be a good idea to check for contiguity during flood fill.

nh_floodfill_assignment.to_csv('Recitation maps/nh_floodfill_map.csv',index=False)


Import the map in DRA. It is based on balancing the number of precints, so the population counts are off. You can adapt the code to take population count into account. You can also start from another district.

Note that the algorithm can break contiguity (by splitting another district), so this should be checked during the traversal. In addition, it has to be applied carefully when you have more than 2 districts.

### 3. From an Existing map - Flip Step

Let's try a random-based strategy which takes a precinct on the border of the two districts and flip it to the other district.
Let's do this 10 times starting from the current official assignment map.

In [ ]:
nh_flipstep_assignment = nh_current_assignment.copy()

## Let's do a simplistic approach. 
## First we randomly select 5 D1 precincts that border D2 and assign them to D2

D1_border_precincts = []

D1_list = nh_latitude_assignment[nh_latitude_assignment['District']==1]['GEOID20']
for precinct in D1_list:
    neighbors = ast.literal_eval(nh_contiguity[nh_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
    for neighbor in neighbors:
        if nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==2:
            #if one of the neighbor is in D2, then this is a border district
            D1_border_precincts.append(precinct)
            break
#Sample 5 and flip them UNLESS it breaks contiguity
flipped_precincts = np.random.choice(D1_border_precincts,5)
for flip in flipped_precincts:
    nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=2
    #check if we broke contiguity and revert the flip if we did
    if(isDistrictContiguous(1,nh_flipstep_assignment,nh_contiguity) is False or isDistrictContiguous(2,nh_flipstep_assignment,nh_contiguity) is False):
        nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=1
        print("Contiguity broken " + flip)
                                                                        

## First we randomly select 5 DIFFERENT D2 precincts that border D1 and assign them to D1

D2_border_precincts = []
D2_list = nh_latitude_assignment[nh_latitude_assignment['District']==2]['GEOID20']
for precinct in D2_list:
    neighbors = ast.literal_eval(nh_contiguity[nh_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
    for neighbor in neighbors:
        if nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==1 and precinct not in flipped_precincts:
            #if one of the neighbor is in D2, then this is a border district. We do not want to re-flip a district we just flipped
            D2_border_precincts.append(precinct)
            break
#Sample 5 and flip them UNLESS it breaks contiguity
flipped_precincts = np.random.choice(D2_border_precincts,5)
for flip in flipped_precincts:
    nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=1
    #check if we broke contiguity and revert the flip if we did
    if(isDistrictContiguous(1,nh_flipstep_assignment,nh_contiguity) is False or isDistrictContiguous(2,nh_flipstep_assignment,nh_contiguity) is False):
        nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=2
        print("Contiguity broken " + flip)
                                                        
    


    

print(isDistrictContiguous(1,nh_flipstep_assignment,nh_contiguity))
print(isDistrictContiguous(2,nh_flipstep_assignment,nh_contiguity))

nh_flipstep_assignment.to_csv('Recitation maps/nh_flipstep_assignment.csv',index=False)


# Compactness
You will also need some function to measure compactness. Here are some examples. You can write your own.

In [ ]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

We can compute the compactness of individual precincts. Here Barrington.

In [ ]:
pp_compactness(Polygon(shape(nh_precinct_boundaries['33003SAND01'])))

In [ ]:
box_reock_compactness(Polygon(shape(nh_precinct_boundaries['33003SAND01'])))

To calculate the compactness of a District we need to create a shape (Multipolygon) that encloses all precincts in the District.

In [ ]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [ ]:
#Compactness of the current assignment
print("D1 PP : "+str(pp_compactness(getDistrictShape(1,nh_current_assignment,nh_precinct_boundaries))))
print("D2 PP : "+str(pp_compactness(getDistrictShape(2,nh_current_assignment,nh_precinct_boundaries))))
print("D1 BR : "+str(box_reock_compactness(getDistrictShape(1,nh_current_assignment,nh_precinct_boundaries))))
print("D2 BR : "+str(box_reock_compactness(getDistrictShape(2,nh_current_assignment,nh_precinct_boundaries))))

In [ ]:
#Compactness of the floodfill assignment
print("D1 PP : "+str(pp_compactness(getDistrictShape(1,nh_floodfill_assignment,nh_precinct_boundaries))))
print("D2 PP : "+str(pp_compactness(getDistrictShape(2,nh_floodfill_assignment,nh_precinct_boundaries))))
print("D1 BR : "+str(box_reock_compactness(getDistrictShape(1,nh_floodfill_assignment,nh_precinct_boundaries))))
print("D2 BR : "+str(box_reock_compactness(getDistrictShape(2,nh_floodfill_assignment,nh_precinct_boundaries))))

In [ ]:
#Compactness of the 43 Latitude assignment
print("D1 PP : "+str(pp_compactness(getDistrictShape(1,nh_latitude43_assignment,nh_precinct_boundaries))))
print("D2 PP : "+str(pp_compactness(getDistrictShape(2,nh_latitude43_assignment,nh_precinct_boundaries))))
print("D1 BR : "+str(box_reock_compactness(getDistrictShape(1,nh_latitude43_assignment,nh_precinct_boundaries))))
print("D2 BR : "+str(box_reock_compactness(getDistrictShape(2,nh_latitude43_assignment,nh_precinct_boundaries))))

### This should give you the information you need to start creating your own maps.
NJ_Redistricing_StartingPoint.ipnyb has an example map for NJ. 
I suggest you test your algorithms on NH first as running times will be slower on a small map. You can of course change the number of districts in your test code you test different approaches. 